##INSTALL NECESSARY LIBRARIES AND VERSIONS

In [1]:
!pip install bitsandbytes transformers==4.49 accelerate peft datasets safetensors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 109.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers==4.49

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 156.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 115.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [8]:
!pip install autoawq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 118.7 MB/s eta 0:00:00
  Created wheel for autoawq: filename=autoawq-0.2.9-py3-none-any.whl size=115106 sha256=cec3d62b47aef7eaab9d5fa7fe4cc0d0c6626798ef888d2a09eb1e04acf0b5b2
  Stored in directory: /root/.cache/pip/wheels/45/1a/7b/7314b3a958454e8ce349f600829a3f0a6a05aeebf987be1e16
Successfully built autoawq
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.0
    Uninstalling transformers-4.37.0:
      Successfully uninstalled transformers-4.37.0


##DOWNLOAD MODEL

In [1]:
#Ok. This works. MAKE SURE TRANSFORMERS >=4.39, <=4.49!
#Use Transformers 4.49 for quantization. It works for both florence2 AND bitsandbytes

#workaround for unnecessary flash_attn requirement
#workaround attribute to James Braza https://huggingface.co/jamesbraza

from unittest.mock import patch
from transformers.dynamic_module_utils import get_imports

from transformers import AutoProcessor, AutoModelForCausalLM
import torch

import os

model_path = "microsoft/Florence-2-base"

def fixed_get_imports(filename: str | os.PathLike) -> list[str]:
    if not str(filename).endswith("modeling_florence2.py"):
        return get_imports(filename)
    imports = get_imports(filename)
    if "flash_attn" in imports:
      imports.remove("flash_attn")
    return imports

with patch("transformers.dynamic_module_utils.get_imports", fixed_get_imports): #workaround for unnecessary flash_attn requirement
    model = AutoModelForCausalLM.from_pretrained(model_path,
                                                 attn_implementation="sdpa",
                                                 torch_dtype=torch.float32,
                                                 trust_remote_code=True,
                                                 force_download = True)
    processor = AutoProcessor.from_pretrained(
      model_path,
      trust_remote_code=True,
      force_download = True
    )
#end attribute to James Braza


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

The repository for microsoft/Florence-2-base contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Florence-2-base.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


##QUANTIZATION

In [1]:
#Quantized with BitsandBytes
#bitsandbytes sucks this doesn't run right

#MAKE SURE TRANSFORMERS == 4.49!

#workaround for unnecessary flash_attn requirement
#workaround attribute to James Braza https://huggingface.co/jamesbraza

from unittest.mock import patch
from transformers.dynamic_module_utils import get_imports
from transformers import AutoProcessor, AutoModelForCausalLM
import torch, os

model_path = "microsoft/Florence-2-base"

def fixed_get_imports(filename: str | os.PathLike) -> list[str]:
    if not str(filename).endswith("modeling_florence2.py"):
        return get_imports(filename)
    imports = get_imports(filename)
    if "flash_attn" in imports:
        imports.remove("flash_attn")
    return imports

# 4bit quantization config
bnb_config = dict(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

with patch("transformers.dynamic_module_utils.get_imports", fixed_get_imports):
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        trust_remote_code=True,
        attn_implementation="sdpa",
        **bnb_config
    )
    processor = AutoProcessor.from_pretrained(
        model_path,
        trust_remote_code=True
    )


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


preprocessor_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

processing_florence2.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

###SAVE QUANTIZATION

In [2]:
#SAVE QUANTIZATION
save_path = "/content/florence2-4bit-optimum"
model.save_pretrained(save_path)
processor.save_pretrained(save_path)

print("Saved to", save_path)

Saved to /content/florence2-4bit-optimum


In [3]:
!zip -r florence2-4bit-bitsnbytes_v1.zip florence2-4bit-optimum

  adding: florence2-4bit-optimum/ (stored 0%)
  adding: florence2-4bit-optimum/preprocessor_config.json (deflated 53%)
  adding: florence2-4bit-optimum/special_tokens_map.json (deflated 98%)
  adding: florence2-4bit-optimum/model.safetensors (deflated 6%)
  adding: florence2-4bit-optimum/merges.txt (deflated 53%)
  adding: florence2-4bit-optimum/config.json (deflated 75%)
  adding: florence2-4bit-optimum/tokenizer.json (deflated 83%)
  adding: florence2-4bit-optimum/tokenizer_config.json (deflated 95%)
  adding: florence2-4bit-optimum/added_tokens.json (deflated 79%)
  adding: florence2-4bit-optimum/generation_config.json (deflated 47%)
  adding: florence2-4bit-optimum/vocab.json (deflated 59%)


In [6]:
from google.colab import files

files.download('florence2-4bit-bitsnbytes_v1.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>